# 🚀 Improved CNN-LSTM Model for Network Intrusion Detection

## CSE-CIC-IDS-2018 Dataset

### 📌 What's Different in This Implementation?

This notebook fixes critical issues in the original CNN-LSTM implementation:

#### ✅ Fixed Issues:
1. **Conv1D instead of Conv2D** - Appropriate for sequential tabular data
2. **Proper sequence creation** - Sliding window with configurable stride
3. **Reduced dropout** - Better learning without over-regularization
4. **Better architecture** - Three variants to test (CNN-LSTM v1, v2, LSTM-only)
5. **Improved training strategy** - Better callbacks and hyperparameters

#### 📊 Expected Performance:
- **Target Accuracy**: 75-85% (vs 51% in original)
- **Target ROC-AUC**: 0.80-0.90 (vs 0.50 in original)
- **Training Time**: ~15-25 minutes on GPU

## 📦 Step 1: Setup and Imports

In [ ]:
# Install required packages (if needed)
# !pip install tensorflow pandas numpy scikit-learn matplotlib seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks, regularizers
from sklearn.metrics import (
    classification_report, confusion_matrix,
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, roc_curve
)
from sklearn.utils.class_weight import compute_class_weight
import time
import pickle
import json
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("IMPROVED CNN-LSTM MODEL FOR INTRUSION DETECTION")
print("="*80)
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU'))} GPU(s)")
if tf.config.list_physical_devices('GPU'):
    print("GPU devices:", tf.config.list_physical_devices('GPU'))
else:
    print("⚠️  No GPU detected. Training will be slower on CPU.")

## ⚙️ Step 2: Configuration

In [ ]:
# Configuration
# UPDATE THESE PATHS TO MATCH YOUR GOOGLE DRIVE STRUCTURE

PROJECT_DIR = '/content/drive/MyDrive/IDS_Research'
MODEL_DIR = f'{PROJECT_DIR}/models'
RESULTS_DIR = f'{PROJECT_DIR}/results'

# Model hyperparameters
TIME_STEPS = 10        # Number of time steps to look back
BATCH_SIZE = 256       # Batch size for training
EPOCHS = 30            # Maximum epochs
LEARNING_RATE = 0.001  # Initial learning rate

# Choose architecture: 'v1', 'v2', or 'lstm_only'
ARCHITECTURE = 'v1'  # Start with v1, can try others later

print("Configuration:")
print(f"  Time steps: {TIME_STEPS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Epochs: {EPOCHS}")
print(f"  Architecture: {ARCHITECTURE}")

## 📊 Step 3: Load Preprocessed Data

**Important:** This notebook assumes you've already run your data preprocessing.

You should have:
- `X_train_scaled`, `y_train`
- `X_val_scaled`, `y_val`  
- `X_test_scaled`, `y_test`

If you haven't done this yet, run your ML_IDS_v4.ipynb notebook up to the preprocessing section first.

In [ ]:
# Check if preprocessed data exists
try:
    print("Checking preprocessed data...")
    print(f"X_train_scaled shape: {X_train_scaled.shape}")
    print(f"X_val_scaled shape: {X_val_scaled.shape}")
    print(f"X_test_scaled shape: {X_test_scaled.shape}")
    print(f"\nLabel distributions:")
    print(f"  Train: {np.bincount(y_train)}")
    print(f"  Val:   {np.bincount(y_val)}")
    print(f"  Test:  {np.bincount(y_test)}")
    print("\n✓ Data loaded successfully!")
except NameError:
    print("❌ Error: Preprocessed data not found!")
    print("\nPlease run your data preprocessing first.")
    print("You can either:")
    print("1. Run ML_IDS_v4.ipynb up to the preprocessing section")
    print("2. Or copy the preprocessing code from that notebook here")
    raise

## 🔄 Step 4: Create Optimized Sequences

This is where we fix the temporal sequence creation!

In [ ]:
def create_optimized_sequences(X, y, time_steps=10, stride=1):
    """
    Create sequences with sliding window approach
    
    Args:
        X: Feature matrix (numpy array or DataFrame)
        y: Labels (pandas Series or numpy array)
        time_steps: Number of time steps to look back
        stride: Step size for sliding window (1=no skip, 5=skip 4)
    
    Returns:
        X_seq: Sequential data (samples, time_steps, features)
        y_seq: Labels for sequences
    """
    print(f"\nCreating sequences with time_steps={time_steps}, stride={stride}...")
    
    # Convert to numpy if needed
    if isinstance(X, pd.DataFrame):
        X = X.values
    if isinstance(y, pd.Series):
        y = y.values
    
    Xs, ys = [], []
    
    # Sliding window approach
    for i in range(0, len(X) - time_steps, stride):
        sequence = X[i:(i + time_steps)]
        label = y[i + time_steps]
        Xs.append(sequence)
        ys.append(label)
    
    X_seq = np.array(Xs)
    y_seq = np.array(ys)
    
    print(f"✓ Created {len(X_seq):,} sequences")
    print(f"  Shape: {X_seq.shape}")
    print(f"  Label distribution: {np.bincount(y_seq.astype(int))}")
    
    return X_seq, y_seq

# Create sequences
# Using stride=5 for train/val to reduce overlap and speed up training
# Using stride=1 for test to evaluate on all data

print("="*80)
print("CREATING SEQUENTIAL DATA")
print("="*80)

X_train_seq, y_train_seq = create_optimized_sequences(
    X_train_scaled, y_train, time_steps=TIME_STEPS, stride=5
)

X_val_seq, y_val_seq = create_optimized_sequences(
    X_val_scaled, y_val, time_steps=TIME_STEPS, stride=5
)

X_test_seq, y_test_seq = create_optimized_sequences(
    X_test_scaled, y_test, time_steps=TIME_STEPS, stride=1
)

print("\n✓ All sequences created successfully!")

## 🏗️ Step 5: Build Improved Model Architecture

In [ ]:
def build_improved_cnn_lstm_v1(input_shape):
    """
    Improved CNN-LSTM using Conv1D
    
    Key improvements:
    - Conv1D instead of Conv2D (appropriate for sequential data)
    - Reduced dropout (was 110%, now 30%)
    - Better regularization with L2
    - Streamlined architecture
    """
    model = models.Sequential([
        # Input
        layers.Input(shape=input_shape),
        
        # CNN Block 1 - Local feature extraction
        layers.Conv1D(64, kernel_size=3, activation='relu', padding='same',
                     kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(0.3),
        
        # CNN Block 2
        layers.Conv1D(128, kernel_size=3, activation='relu', padding='same',
                     kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        
        # LSTM Block - Temporal dependencies
        layers.LSTM(128, return_sequences=True,
                   kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.3),
        
        layers.LSTM(64, return_sequences=False,
                   kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.3),
        
        # Classification layers
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.1),
        
        # Output
        layers.Dense(1, activation='sigmoid')
    ])
    
    return model


def build_improved_cnn_lstm_v2(input_shape):
    """
    Simplified CNN-LSTM - lighter architecture
    """
    model = models.Sequential([
        layers.Input(shape=input_shape),
        
        # Simple CNN
        layers.Conv1D(64, kernel_size=3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        
        # LSTM
        layers.LSTM(128, return_sequences=True),
        layers.Dropout(0.3),
        layers.LSTM(64, return_sequences=False),
        
        # Classification
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    
    return model


def build_lstm_only(input_shape):
    """
    LSTM-only model (no CNN)
    May perform better on tabular data
    """
    model = models.Sequential([
        layers.Input(shape=input_shape),
        
        # Bidirectional LSTM
        layers.Bidirectional(layers.LSTM(128, return_sequences=True)),
        layers.Dropout(0.3),
        
        layers.Bidirectional(layers.LSTM(64, return_sequences=False)),
        layers.Dropout(0.3),
        
        # Classification
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    
    return model


# Build selected architecture
print("="*80)
print("BUILDING MODEL")
print("="*80)

input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
print(f"Input shape: {input_shape}")
print(f"Architecture: {ARCHITECTURE}")

if ARCHITECTURE == 'v1':
    model = build_improved_cnn_lstm_v1(input_shape)
elif ARCHITECTURE == 'v2':
    model = build_improved_cnn_lstm_v2(input_shape)
elif ARCHITECTURE == 'lstm_only':
    model = build_lstm_only(input_shape)
else:
    raise ValueError(f"Unknown architecture: {ARCHITECTURE}")

model.summary()

## 🎯 Step 6: Compile Model with Class Weights

In [ ]:
# Calculate class weights for imbalanced data
class_weights_array = compute_class_weight(
    'balanced',
    classes=np.unique(y_train_seq),
    y=y_train_seq
)
class_weights = {
    0: class_weights_array[0],
    1: class_weights_array[1]
}

print(f"Class weights: {class_weights}")
print(f"This helps handle imbalanced data (more benign than attack samples)")

# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc')
    ]
)

print("\n✓ Model compiled successfully!")

## 🔧 Step 7: Setup Training Callbacks

In [ ]:
# Create callbacks for better training
callbacks_list = [
    # Early stopping - stop if no improvement
    callbacks.EarlyStopping(
        monitor='val_auc',
        patience=7,
        restore_best_weights=True,
        mode='max',
        verbose=1
    ),
    
    # Reduce learning rate when stuck
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    
    # Save best model
    callbacks.ModelCheckpoint(
        f'{MODEL_DIR}/improved_cnn_lstm_best.h5',
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    
    # TensorBoard logging
    callbacks.TensorBoard(
        log_dir=f'{PROJECT_DIR}/logs',
        histogram_freq=1
    )
]

print("✓ Callbacks configured:")
print("  - Early stopping (patience=7)")
print("  - Learning rate reduction (patience=3)")
print("  - Model checkpoint (save best)")
print("  - TensorBoard logging")

## 🚀 Step 8: Train Model

This will take 15-25 minutes on GPU, longer on CPU.

In [ ]:
print("="*80)
print("TRAINING MODEL")
print("="*80)
print(f"Epochs: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Training samples: {len(X_train_seq):,}")
print(f"Validation samples: {len(X_val_seq):,}")
print("\nTraining started...")

start_time = time.time()

history = model.fit(
    X_train_seq, y_train_seq,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val_seq, y_val_seq),
    class_weight=class_weights,
    callbacks=callbacks_list,
    verbose=1
)

training_time = time.time() - start_time

print("\n" + "="*80)
print(f"✓ TRAINING COMPLETED!")
print("="*80)
print(f"Total time: {training_time:.2f} seconds ({training_time/60:.2f} minutes)")
print(f"Final training accuracy: {history.history['accuracy'][-1]*100:.2f}%")
print(f"Final validation accuracy: {history.history['val_accuracy'][-1]*100:.2f}%")
print(f"Best validation AUC: {max(history.history['val_auc']):.4f}")

## 📊 Step 9: Visualize Training History

In [ ]:
# Plot comprehensive training history
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Improved CNN-LSTM Training History', fontsize=16, fontweight='bold')

metrics = ['loss', 'accuracy', 'precision', 'recall', 'auc']
titles = ['Loss', 'Accuracy', 'Precision', 'Recall', 'AUC']

for idx, (metric, title) in enumerate(zip(metrics, titles)):
    ax = axes[idx // 3, idx % 3]
    
    ax.plot(history.history[metric], label=f'Training {title}', linewidth=2)
    ax.plot(history.history[f'val_{metric}'], label=f'Validation {title}', linewidth=2)
    ax.set_title(f'Model {title}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Epoch')
    ax.set_ylabel(title)
    ax.legend()
    ax.grid(True, alpha=0.3)

# Learning rate plot
if 'lr' in history.history:
    ax = axes[1, 2]
    ax.plot(history.history['lr'], label='Learning Rate', linewidth=2, color='red')
    ax.set_title('Learning Rate Schedule', fontsize=12, fontweight='bold')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Learning Rate')
    ax.set_yscale('log')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/improved_cnn_lstm_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Training history visualized and saved!")

## 🎯 Step 10: Evaluate on Test Set

In [ ]:
print("="*80)
print("TEST SET EVALUATION")
print("="*80)

# Make predictions
start_time = time.time()
y_pred_proba = model.predict(X_test_seq, verbose=0)
inference_time = time.time() - start_time

y_pred = (y_pred_proba > 0.5).astype(int).flatten()
y_pred_proba = y_pred_proba.flatten()

# Calculate metrics
accuracy = accuracy_score(y_test_seq, y_pred)
precision = precision_score(y_test_seq, y_pred, zero_division=0)
recall = recall_score(y_test_seq, y_pred, zero_division=0)
f1 = f1_score(y_test_seq, y_pred, zero_division=0)
roc_auc = roc_auc_score(y_test_seq, y_pred_proba)

# Confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test_seq, y_pred).ravel()
fpr_value = fp / (fp + tn) if (fp + tn) > 0 else 0

# Latency
avg_latency = (inference_time / len(X_test_seq)) * 1000  # ms

print(f"\nImproved CNN-LSTM Test Performance:")
print(f"  Accuracy:  {accuracy*100:.2f}%")
print(f"  Precision: {precision*100:.2f}%")
print(f"  Recall:    {recall*100:.2f}%")
print(f"  F1-Score:  {f1*100:.2f}%")
print(f"  ROC-AUC:   {roc_auc:.4f}")
print(f"  FPR:       {fpr_value*100:.2f}%")

print(f"\nConfusion Matrix:")
print(f"  TN: {tn:>6,}  FP: {fp:>6,}")
print(f"  FN: {fn:>6,}  TP: {tp:>6,}")

print(f"\nInference Performance:")
print(f"  Avg Latency: {avg_latency:.2f} ms/sample")
print(f"  Total Time:  {inference_time:.2f} seconds")
print(f"  Throughput:  {len(X_test_seq)/inference_time:.2f} samples/sec")

# Comparison with original
print(f"\n" + "="*80)
print("COMPARISON WITH ORIGINAL CNN-LSTM")
print("="*80)
original_acc = 0.510
original_auc = 0.501
improvement_acc = ((accuracy - original_acc) / original_acc) * 100
improvement_auc = ((roc_auc - original_auc) / original_auc) * 100

print(f"  Accuracy:  {original_acc*100:.2f}% → {accuracy*100:.2f}% ({improvement_acc:+.1f}% improvement)")
print(f"  ROC-AUC:   {original_auc:.3f} → {roc_auc:.3f} ({improvement_auc:+.1f}% improvement)")

# Store results
results = {
    'accuracy': float(accuracy),
    'precision': float(precision),
    'recall': float(recall),
    'f1_score': float(f1),
    'roc_auc': float(roc_auc),
    'fpr': float(fpr_value),
    'confusion_matrix': {'tn': int(tn), 'fp': int(fp), 'fn': int(fn), 'tp': int(tp)},
    'avg_latency_ms': float(avg_latency),
    'training_time_seconds': float(training_time)
}

## 📈 Step 11: Visualize Results

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test_seq, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Benign', 'Attack'],
            yticklabels=['Benign', 'Attack'])
plt.title('Improved CNN-LSTM Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/improved_cnn_lstm_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# ROC Curve
fpr, tpr, _ = roc_curve(y_test_seq, y_pred_proba)
plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='blue', lw=2.5,
         label=f'Improved CNN-LSTM (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve - Improved CNN-LSTM', fontsize=14, fontweight='bold')
plt.legend(loc="lower right", fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/improved_cnn_lstm_roc_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Visualizations saved!")

## 💾 Step 12: Save Model and Results

In [ ]:
print("="*80)
print("SAVING MODEL AND RESULTS")
print("="*80)

# Save final model
model_path = f'{MODEL_DIR}/improved_cnn_lstm_ids_model.h5'
model.save(model_path)
print(f"✓ Model saved to: {model_path}")

# Save results as JSON
results_dict = {
    'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
    'dataset': 'CSE-CIC-IDS-2018',
    'architecture': ARCHITECTURE,
    'time_steps': TIME_STEPS,
    'config': {
        'batch_size': BATCH_SIZE,
        'epochs': EPOCHS,
        'learning_rate': LEARNING_RATE,
    },
    'results': results,
    'training_samples': int(len(X_train_seq)),
    'validation_samples': int(len(X_val_seq)),
    'test_samples': int(len(X_test_seq)),
    'comparison_with_original': {
        'original_accuracy': 0.510,
        'improved_accuracy': float(accuracy),
        'accuracy_improvement_percent': float(improvement_acc),
        'original_auc': 0.501,
        'improved_auc': float(roc_auc),
        'auc_improvement_percent': float(improvement_auc)
    }
}

results_path = f'{RESULTS_DIR}/improved_cnn_lstm_results.json'
with open(results_path, 'w') as f:
    json.dump(results_dict, f, indent=4)

print(f"✓ Results saved to: {results_path}")

print("\n" + "="*80)
print("✅ ALL DONE!")
print("="*80)
print("\nYour improved CNN-LSTM model is ready for deployment!")
print(f"\nKey files saved:")
print(f"  1. Model: {model_path}")
print(f"  2. Results: {results_path}")
print(f"  3. Visualizations: {RESULTS_DIR}/improved_cnn_lstm_*.png")

## 🔬 Step 13: Optional - Compare All Three Architectures

Run this cell to test all three architectures and compare them!

In [ ]:
# Uncomment to run full comparison

# print("="*80)
# print("COMPARING ALL ARCHITECTURES")
# print("="*80)

# architectures = ['v1', 'v2', 'lstm_only']
# comparison_results = {}

# for arch in architectures:
#     print(f"\n{'='*80}")
#     print(f"Testing: {arch}")
#     print(f"{'='*80}")
#     
#     # Build model
#     if arch == 'v1':
#         test_model = build_improved_cnn_lstm_v1(input_shape)
#     elif arch == 'v2':
#         test_model = build_improved_cnn_lstm_v2(input_shape)
#     else:
#         test_model = build_lstm_only(input_shape)
#     
#     # Compile
#     test_model.compile(
#         optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
#         loss='binary_crossentropy',
#         metrics=['accuracy', keras.metrics.AUC(name='auc')]
#     )
#     
#     # Train
#     hist = test_model.fit(
#         X_train_seq, y_train_seq,
#         batch_size=BATCH_SIZE,
#         epochs=20,  # Reduced epochs for comparison
#         validation_data=(X_val_seq, y_val_seq),
#         class_weight=class_weights,
#         callbacks=[callbacks.EarlyStopping(monitor='val_auc', patience=5, restore_best_weights=True)],
#         verbose=0
#     )
#     
#     # Evaluate
#     y_pred_proba = test_model.predict(X_test_seq, verbose=0)
#     y_pred = (y_pred_proba > 0.5).astype(int).flatten()
#     
#     acc = accuracy_score(y_test_seq, y_pred)
#     auc = roc_auc_score(y_test_seq, y_pred_proba)
#     
#     comparison_results[arch] = {'accuracy': acc, 'auc': auc}
#     print(f"  Accuracy: {acc*100:.2f}%, AUC: {auc:.4f}")

# # Plot comparison
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
# 
# archs = list(comparison_results.keys())
# accs = [comparison_results[a]['accuracy'] for a in archs]
# aucs = [comparison_results[a]['auc'] for a in archs]
# 
# ax1.bar(archs, accs, color=['blue', 'green', 'orange'])
# ax1.set_title('Accuracy Comparison')
# ax1.set_ylabel('Accuracy')
# ax1.set_ylim([0, 1])
# 
# ax2.bar(archs, aucs, color=['blue', 'green', 'orange'])
# ax2.set_title('AUC Comparison')
# ax2.set_ylabel('AUC')
# ax2.set_ylim([0, 1])
# 
# plt.tight_layout()
# plt.savefig(f'{RESULTS_DIR}/architecture_comparison.png', dpi=300)
# plt.show()
# 
# print("\n✓ Architecture comparison complete!")